# Clasificación de Texto con scikit-learn

Basado en: http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

Este tutorial es sobre clasificación de texto pero bien puede servir como una rápida introducción general a scikit-learn.

## Corpus

Primero cargamos un corpus de categorización de texto. En nuestro caso, usamos un subconjunto del corpus "20 newsgroups", que se compone de 2257 posts en foros de cuatro temáticas diferentes (ateísmo, cristianismo, gráficos de computadora y medicina):

In [27]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
len(twenty_train.data), len(twenty_train.target)

(2257, 2257)

In [9]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

Veamos por ejemplo el primer documento:

In [4]:
print(twenty_train.data[0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



Vemos que este documento corresponde al tema de gráficos de computadora ('comp.graphics'):

In [8]:
twenty_train.target[0]
twenty_train.target_names[twenty_train.target[0]]

'comp.graphics'

## Features: Bag of Words

- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Los features van a ser las palabras de los posts (*bag-of-words*). Más precisamente, vamos a contar la frecuencia de aparición de cada palabra en cada documento. Para eso podemos usar la clase `CountVectorizer`:

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

X_train_counts tiene una fila por cada documento (post) y una columna por cada feature. Podemos consultar a qué índice corresponde cada palabra:

In [13]:
features = count_vect.get_feature_names()
features.index('converting')

9805

Acá vemos que la palabra 'converting' tiene índice 9805. Podemos confirmar que el conteo de la palabra 'converting' para el primer documento es 2 (se cuenta dos veces porque CountVectorizer ignora mayúsculas por defecto):

In [14]:
X_train_counts[0,9805]

2

## Matrices Dispersas (o Ralas)

De las ~36000 palabras que observamos en la totalidad de los documentos, sólo muy pocas van a aparecer en cada documento individualmente. Esto quiere decir que la gran mayoría de las entradas de la matriz de conteo van a ser ceros. Este tipo de matrices se llaman matrices esparsas, dispersas o ralas (*sparse matrices* en inglés).

Por suerte, existen implementaciones de matrices dispersas que no guardan explícitamente todas las entradas de la matriz si no sólo las que son distintas de cero.

Scipy trae varias versiones de matrices esparsas, y el `CountVectorizer` devuelve este tipo de matrices:

In [47]:
X_train_counts

<2257x35788 sparse matrix of type '<class 'numpy.int64'>'
	with 365886 stored elements in Compressed Sparse Row format>

Vemos que la matriz es 2257x35788 y tiene 365886 elementos distintos de cero. Haciendo la cuenta vemos que sólo un 0.4% de las entradas de la matriz son distintas de cero:

In [48]:
365886.0 / (2257*35788)

0.004529776814469733

## Features: Preprocesamiento

- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
- https://en.wikipedia.org/wiki/Tf%E2%80%93idf

Los features así como están tienen dos problemas:

1. Los documentos más largos van a tener más counts para todos los features.
2. Las palabras más frecuentes (las funcionales, como los artículos y las preposiciones) son poco informativas ya que aparecen mucho en todos los temas.

Estos dos problemas meten ruido en los features que hace más difícil que sobresalgan los features realmente informativos. La solución es hacer un *downscaling* de ambas cosas, llamado [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). Scikit-learn provee el `TfidfTransformer` para hacerlo:

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

Podemos ver, por ejemplo, que el conteo para la palabra 'converting' en el primer documento ahora está normalizado (antes valía 2):

In [17]:
X_train_tfidf[0,9805]

0.21567205914741702

## Clasificador Multinomial Naive-Bayes

- http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes

Ahora que ya tenemos los features con el debido preprocesamiento, podemos instanciar y entrenar un clasificador:

In [19]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_tfidf, twenty_train.target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Como pueden ver, para el entrenamiento pasamos como parámetro una matriz con los vectores de features para cada documento (uno por fila) y la lista de etiquetas de los documentos.

Una vez entrenado el clasificador, se puede utilizar para predecir la clasificación de nuevos documentos. Por supuesto, antes se debe pasar los documentos por el mismo preprocesamiento que se usó al entrenar:

In [21]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
predicted

array([3, 1])

In [22]:
twenty_train.target_names[3], twenty_train.target_names[1]

('soc.religion.christian', 'comp.graphics')

Claramente 'God is love' se clasifica como cristianismo, y 'OpenGL ...' como gráficos de computadoras. Podemos probar con cosas más raras:

In [33]:
docs_new = ['God is a computer', 'God is computer graphics']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
[twenty_train.target_names[p] for p in predicted]

['soc.religion.christian', 'comp.graphics']

Vemos que 'god' favorece mucho más a la clase 'christian' que 'computer' a la clase 'comp.graphics', pero al agregar 'graphics' termina siendo elegida 'comp.graphics'.

## El Multinomial Naive-Bayes por Dentro

Los parámetros de un modelo MNB son los siguientes:

1. Probabilidad a priori de cada clase (prior): p(c).
2. Probabilidad de un feature dada una clase: p(f|c).

A continuación vemos cada una de las dos.

### Probabilidad de clase

En scikit-learn se guardan las log-probabilidades en el atributo `class_log_prior_`:

In [39]:
from numpy import exp
list(zip(twenty_train.target_names, exp(clf.class_log_prior_)))

[('alt.atheism', 0.21267168808152423),
 ('comp.graphics', 0.25875055383252116),
 ('sci.med', 0.26318121400088634),
 ('soc.religion.christian', 0.26539654408506874)]

Acá podemos ver que a priori la clase más probable es 'christian'. Esto se debe a que la mayoría de los documentos que se usaron para entrenar pertenecen a esta clase.

### Probabilidad de feature

En scikit-learn se guardan las log-probabilidades en el atributo `feature_log_prob_`, que es una matriz (m, n) a donde m es la cantidad de clases y n la cantidad de features.

Por ejemplo, podemos preguntar la probabilidad de la palabra 'god' para cada una de las clases viendo en la columna correspondiente a ese feature:

In [43]:
i = features.index('god')
list(zip(twenty_train.target_names, exp(clf.feature_log_prob_[:,i])))

[('alt.atheism', 0.00037269074626827755),
 ('comp.graphics', 3.2614957950227121e-05),
 ('sci.med', 2.6122286434412079e-05),
 ('soc.religion.christian', 0.0007471829003996937)]

Acá se puede ver claramente que 'god' es mucho más probable en 'christian' y 'atheism' que en 'graphics' y 'med'.

Veamos también las probabilidades para 'computer':

In [46]:
i = features.index('computer')
list(zip(twenty_train.target_names, exp(clf.feature_log_prob_[:,i])))

[('alt.atheism', 6.3638553439577644e-05),
 ('comp.graphics', 0.00018187889710043111),
 ('sci.med', 0.00016266121829935505),
 ('soc.religion.christian', 5.5180221237835211e-05)]

Acá vemos cómo 'computer' favorece 'graphics' y 'med' sobre 'atheism' y 'christian', pero también vemos que las priobabilidades no son tan altas como las de 'god'.

# Ejercicios

1. Terminar el [tutorial original](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).
2. Probar el clasificador sin hacer TF-IDF y ver qué da.
2. Leer y tratar de entender el código fuente de la clase [MultinomialNB](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/naive_bayes.py#L562).
